
# Double pendulum

In [ ]:
# ] add OrdinaryDiffEqTsit5

In [ ]:
# ] add Printf

In [ ]:
# ] add Colors

In [ ]:

using OrdinaryDiffEqTsit5

In [ ]:

using Printf

In [ ]:

using PyPlot

In [ ]:

using Colors

In [ ]:

"""
    
    doublependulum!(dudt, u, p, t)

.......
"""
function doublependulum!(dudt, u, p, t)
    # unpack variables and parameters
    theta1 = u[1]
    theta2 = u[2]
    dtheta1dt = u[3]
    dtheta2dt = u[4]
    κ, ν = p
    
    sn, cs = sincos(theta1 - theta2)
    a1 = κ*cs
    a2 = ν*cs
    γ = 1/(1.0 - a1*a2)
    
    f1 = -κ*sn*dtheta2dt^2 - sin(theta1)
    f2 =  ν*(sn*dtheta1dt^2 - ν*sin(theta2))

    dudt[1] = u[3]
    dudt[2] = u[4]
    dudt[3] = γ*(f1 - a1*f2)
    dudt[4] = γ*(f2 - a2*f1)
    
    return nothing
end

In [ ]:

l1 = 1.0  # length of pendulum1
l2 = 1.0  # length of pendulum2
m1 = 1.0  # mass of pendulum1
m2 = 1.0; # mass of pendulum2

In [ ]:

ν = l1/l2
κ = 1 / ν * m2 / (m1 + m2)
# Pack the parameters
p = (κ, ν);

In [ ]:

tspan = (0.0, 50.0)
initial = [pi, pi/100, 0.0, 0.0];

In [ ]:

prob = ODEProblem(doublependulum!, initial, tspan, p);

In [ ]:

sol = solve(prob, Tsit5(), abstol=1e-7, reltol=1e-7);

In [ ]:

np = 2001
t = range(tspan[1], tspan[2], np)
sl = sol(t);

In [ ]:

theta1 = sl[1, :]
theta2 = sl[2, :];

In [ ]:

x1 =  l1 .* sin.(theta1)
y1 = -l1 .* cos.(theta1)
x2 = x1 .+ l2 .* sin.(theta2)
y2 = y1 .- l2 .* cos.(theta2);

In [ ]:

function snapshot_ft(nt, pts_disp, box, t, x1, y1, x2, y2)
    nt_start = max(1, nt-pts_disp)
    xl, xr, yb, yt = box
    axis("square")
    xlim(xl, xr)
    ylim(yb, yt)
    titl = @sprintf("Double pendulum t = %5.2f .. %5.2f",
                    t[nt_start], t[nt])
    title(titl, family="monospace")
    xlabel(L"$x$")
    ylabel(L"$y$")
    grid(true)

    # pivot
    plot(0.0, 0.0, color="black", marker="x", markersize=5.0)

    # two rods
    plot([0.0, x1[nt],x2[nt]], [0.0, y1[nt], y2[nt]], color="black")

    # mass 1
    c1 = "green"
    plot(x1[nt], y1[nt], color=c1, marker="o", markersize=5.0)
    
    # mass 2
    c2 = "blue"
    plot(x2[nt], y2[nt], color=c2, marker="o", markersize=5.0)

    # fading tail trajectory of mass 2
    c2p = parse(RGBA, c2)
    for i = nt_start:nt-1
        transp = 1.0 - ((nt-i-1)/(pts_disp-1))^2
        cc = (c2p.r, c2p.g, c2p.b, transp)
        plot([x2[i], x2[i+1]], [y2[i], y2[i+1]], color=cc, linewidth=0.5)
    end
    return nothing
end;

In [ ]:

axis_lim = (l1 + l2)*1.2
xl = -axis_lim
xr = -xl
yb = -axis_lim
yt = -yb
box = (xl, xr, yb, yt);

In [ ]:

# Define the length of the trajectory's tail
f = 0.1                      # Fraction of displayed points
pts_disp = round(Int, f*np)  # Number of displayed points

In [ ]:

fig = figure()
for nt = 1:10:np
    snapshot_ft(nt, pts_disp, box, t, x1, y1, x2, y2)
    display(fig)
    IJulia.clear_output(true)
    clf()
end


# Linear regression

In [ ]:

"""
    alpha, beta, sigma = linear_regression(x, y)

Least square fit y = alpha + beta x. sigma is the standard error of beta
"""
function linear_regression(x, y)
    np = length(x)
    xbar = sum(x)/np
    ybar = sum(y)/np
    x2 = sum((xbar .- x) .^2)
    beta = sum((xbar .- x) .* (ybar .- y)) / x2
    alpha = ybar - beta * xbar
    sigma = sqrt(sum((y .- alpha .- beta .* x) .^ 2)/((np - 2)*x2))
    return alpha, beta, sigma
end

Test the code:

In [ ]:

xmin = 0.0
xmax = 10.0
np = 200
sc = 1.5
x = range(xmin, xmax, np)
y = 2 .* x .+ sc .* randn(np);

In [ ]:

alpha1, beta1, sigma1 = linear_regression(x, y)

In [ ]:

using PyPlot

In [ ]:

plot(x, y, linestyle="none", marker=".", label="noisy data")
plot(x, alpha1 .+ beta1 .* x, linestyle="solid", label="linear lsq fit")

grid(true)
xlabel("x")
ylabel("y")
title("Linear regression")
legend();